In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import time

In [ ]:
# list_url= "https://www.upwork.com/nx/search/talent/?q=data%20analyst"
# # https://www.upwork.com/nx/search/talent/?q=data%20analyst&page=1

# response= requests.get(list_url, headers = {'User-agent': 'Chrome/129.0.6668.101'})
# print(response.status_code)
# list_soup= BeautifulSoup(response.text, "html.parser")
# profiles_section= list_soup.find("section", {"data-ev-label": "search_results"})
# profiles= list_soup.find_all("article")

In [9]:
base_url = "https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing"

In [10]:
num_pages = 20

In [11]:
profile_details_list=[]

In [12]:
for page in range(1, num_pages + 1):
    # Construct the URL for the current page
    url = f"{base_url}&page={page}"
    print(f"Scraping URL: {url}")  # Print the URL being scraped

    # Send a GET request to the URL
    response = requests.get(url, headers={'User-agent': 'Chrome/129.0.6668.101'})
    print(f"Status Code: {response.status_code}")
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        list_soup = BeautifulSoup(response.text, "html.parser")

        # Find the section containing profiles
        profiles = list_soup.find_all("article")
        
        # Loop through each profile and extract details
        for prof in profiles:
            # Create a dictionary for each profile
            profile_data = {}

            # Get the profile ID and add it to the dictionary
            profile_id = prof.get("data-ev-contractor_uid")
            profile_data['profile_id'] = profile_id

            # Find the anchor tag with the profile link
            profile_link_tag = prof.find("a", class_="profile-link")
            if profile_link_tag:
                # Get the href attribute and construct the full URL
                profile_link = "https://www.upwork.com" + profile_link_tag['href']
                profile_data['profile_link'] = profile_link

            # Add the profile data dictionary to the list
            profile_details_list.append(profile_data)

    else:
        print(f"Failed to retrieve page {page}: Status code {response.status_code}")

# Output the total number of profiles scraped
print(f"Total profiles scraped: {len(profile_details_list)}")

Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=1
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=2
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=3
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=4
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=5
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=6
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=7
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=8
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=9
Status Code: 200
Scraping URL: https://www.upwork.com/nx/search/talent/?nbs=1&q=marketing&page=10
Status Code: 200
Scraping URL: https://www.upw

In [ ]:
# for prof in profiles:
#     # Create a dictionary for each profile
#     profile_data = {}
    
#     # Get the profile ID and add it to the dictionary
#     profile_id = prof.get("data-ev-contractor_uid")
#     profile_data['profile_id'] = profile_id

#     # Find the anchor tag with the profile link
#     profile_link_tag = prof.find("a", class_="profile-link")
#     if profile_link_tag:
#         # Get the href attribute and construct the full URL
#         profile_link = "https://www.upwork.com" + profile_link_tag['href']
#         profile_data['profile_link'] = profile_link

#     # Add the profile data dictionary to the list
#     profile_details_list.append(profile_data)

#### What do I want to scrape

    Role
    payperhour
    summary 
    
    location
    job sucess perc
    
    total jobs 
    total hours
    
    work history
    skills
    
    Total earnings????

In [13]:
worker_profiles = []
profile_count = 0  # Initialize a counter for profiles

for prof in profile_details_list:
    profile_count += 1  # Increment the profile counter
    print(f"Scraping profile {profile_count}...")  # Print the current profile count
    
    worker_profile = {}
    profile_url = prof['profile_link']
    profile_id = prof['profile_id']
    
    # Extract text from full profile page
    prof_response = requests.get(profile_url, headers={'User-agent': 'Chrome/129.0.6668.101'})
#     print("Scraping:", profile_url)
    print("Status Code:", prof_response.status_code)
    
    if prof_response.status_code == 200:
        prof_soup = BeautifulSoup(prof_response.text, "html.parser")
        profile_details = prof_soup.find("div", {"data-qa-profile-viewer-uid": profile_id})
        
        if profile_details:
            # Extract fields with try-except for error handling
            try:
                worker_profile['title'] = profile_details.find("h2", {"class": "mb-0 pt-lg-2x h4"}).text.strip()
            except AttributeError:
                worker_profile['title'] = None
            
            try:
                worker_profile['pp'] = profile_details.find("h3", {"class": "d-inline h5 nowrap"}).find("span").text.strip()
            except AttributeError:
                worker_profile['pp'] = None
            
            try:
                worker_profile['summary'] = profile_details.find("span", {"class": "text-pre-line break"}).text.strip()
            except AttributeError:
                worker_profile['summary'] = None
            
            try:
                worker_profile['total_jobs'] = profile_details.find("div", {"class": "stat-amount h5"}).find("span").text.strip()
            except AttributeError:
                worker_profile['total_jobs'] = None
            
            try:
                worker_profile['total_hours'] = profile_details.find("div", {"class": "stat-amount h5"}).find("span").text.strip()
            except AttributeError:
                worker_profile['total_hours'] = None
            
            try:
                worker_profile['locality'] = profile_details.find("div", {"class": "d-block location-status text-light-on-inverse"}).find("span", {"itemprop": "locality", "class": "d-inline-block vertical-align-middle ellipsis"}).text.strip()
            except AttributeError:
                worker_profile['locality'] = None
            
            try:
                worker_profile['country'] = profile_details.find("div", {"class": "d-block location-status text-light-on-inverse"}).find("span", {"itemprop": "country-name", "class": "d-inline-block vertical-align-middle ellipsis"}).text.strip()
            except AttributeError:
                worker_profile['country'] = None
            
            try:
                worker_profile['perc_job_success'] = profile_details.find("div", {"class": "cfe-ui-profile-job-success"}).find("span").text.split('%')[0].strip()
            except (AttributeError, IndexError):
                worker_profile['perc_job_success'] = None
            
            # Extract stats with error handling
            stats = profile_details.find_all("div", class_="col-compact min-with-75")
            for stat in stats:
                try:
                    val = stat.find("div", class_="stat-amount").text.strip()
                    label = stat.find("span", class_="text-base-sm text-light-on-inverse").text.strip()
                    if label == "Total jobs":
                        worker_profile['total_jobs'] = val
                    elif label == "Total hours":
                        worker_profile['total_hours'] = val
                except AttributeError:
                    continue
            
            # Extract projects
            projects = []
            try:
                projects_section = profile_details.find("div", {"class": "work-history"}).find_all("h5")
                for proj in projects_section:
                    project= proj.text.strip()
                    projects.append(project)

            except AttributeError:
                pass

            worker_profile['projects'] = projects
            
            # Extract skills
            skills_list = []
            try:
                skills = profile_details.find("div", {"class": "skills"}).find_all("li")
                for skill in skills:
                    for skill_tag in skill.find_all(class_="skill-name"):
                        skill_text = skill_tag.get_text(strip=True)
                        skills_list.append(skill_text)
            except AttributeError:
                pass
            
            worker_profile['skills_list'] = skills_list
            
            # Add the complete profile to the main list
            worker_profiles.append(worker_profile)

Scraping profile 1...
Status Code: 200
Scraping profile 2...
Status Code: 200
Scraping profile 3...
Status Code: 200
Scraping profile 4...
Status Code: 200
Scraping profile 5...
Status Code: 200
Scraping profile 6...
Status Code: 200
Scraping profile 7...
Status Code: 200
Scraping profile 8...
Status Code: 200
Scraping profile 9...
Status Code: 200
Scraping profile 10...
Status Code: 200
Scraping profile 11...
Status Code: 200
Scraping profile 12...
Status Code: 200
Scraping profile 13...
Status Code: 200
Scraping profile 14...
Status Code: 200
Scraping profile 15...
Status Code: 200
Scraping profile 16...
Status Code: 200
Scraping profile 17...
Status Code: 200
Scraping profile 18...
Status Code: 200
Scraping profile 19...
Status Code: 200
Scraping profile 20...
Status Code: 200
Scraping profile 21...
Status Code: 200
Scraping profile 22...
Status Code: 200
Scraping profile 23...
Status Code: 200
Scraping profile 24...
Status Code: 200
Scraping profile 25...
Status Code: 200
Scraping 

In [14]:
worker_profiles= pd.DataFrame(worker_profiles)
worker_profiles

,title,pp,summary,total_jobs,total_hours,locality,country,perc_job_success,projects,skills_list
0,"Fractional CMO, Marketing Strategist, Entrepre...",$299.00/hr,Highest-rated fractional CMO on Upwork🏆. I've ...,404,"1,226",Santa Rosa,None,97,"[30 minute consultation, 30 minute consultatio...","[Startup Consulting, Entrepreneurship, Public ..."
1,Marketing Sales Artificial Intelligence - One ...,$199.00/hr,Top 1% Upwork EXPERT-VETTED Marketing Sales Ar...,840,3,Spring,None,99,"[60 minute consultation, 60 minute consultatio...","[Marketing Strategy, Sales & Marketing, Social..."
2,"SEO, Social Media, Digital Marketing, Web App ...",$10.00/hr,Highly Recommended by Several Upwork Clients.\...,282,"17,516",Mohali,India,98,"[30 minute consultation, Digital Marketer - So...","[Social Media Marketing, Google Ads, Email Mar..."
3,Digital Marketing Expert/ Strategist,$51.00/hr,Creating impact in the digital world isn’t jus...,125,"11,777",Yerevan,Armenia,100,[You will get a Customized Digital Marketing S...,"[Canva, Social Media Management, Social Media ..."
4,Fractional CMO | Marketing & Brand Consultant ...,$75.00/hr,🏆Upwork 1% Expert-Vetted Talent \n\nAs a Marke...,18,673,Aventura,None,100,"[60 minute consultation, Social Media Manageme...","[Social Media Management, Graphic Design, Soci..."
...,...,...,...,...,...,...,...,...,...,...
193,"Clickfunnels, Kajabi, Kartra, GoHighLevel Expe...",$38.00/hr,"I have helped digital marketing agencies, cour...",323,"8,800",Quezon City,Philippines,94,[Seeking a WOW-Factor Kajabi Designer for High...,"[Email Marketing, Keap, ActiveCampaign, WordPr..."
194,SEO Specialist | Digital Marketing Strategist ...,$13.00/hr,🚀 SEO Expert - Secure Guaranteed #1 Google Ran...,244,"18,905",Mohali,India,88,"[Backlinks SEO, SEO Semrush and Yoast expert, ...","[Facebook Advertising, Social Media Marketing,..."
195,"Top Rated Plus - Social Media Manager, Content...",$10.00/hr,⭐⭐⭐⭐⭐Top Rated Plus Expert\n🔶🔶Specialized in D...,113,"13,121",Mohali,India,90,"[Facebook Meta Ads, Facebook/Meta Ads Setup, C...","[Facebook Advertising, Social Media Marketing,..."
196,Copywriting Expert | Turning Customer Insights...,$119.00/hr,"Hi, I’m Kelly—a seasoned copywriter who helps ...",722,"4,915",Tbilisi,Georgia,100,[Our website needs content/copywriting help to...,"[Sales & Marketing Collateral, White Paper, Ad..."


In [15]:
worker_profiles.to_csv("worker_profiles_MKT.csv")